<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">Thierry Parmentelat &amp; Arnaud Legout&nbsp;<img src="media/inria-25-alpha.png" style="display:inline"></span><br/>

# Itérateur et performances

XXX couper en deux pour faire 

* un truc sur la mesure de performances 
* un truc sur les itérateurs

 Voir aussi `ONGOING-py3.md`

## Complément - niveau basique

Dans ce complément, nous allons voir pourquoi il est bien souvent preférable d'utiliser un itérateur comme sujet d'une boucle `for`, plutôt que d'itérer sur une énumération explicite comme une liste.

### Calculs non-instantanés dans un notebook

Si en manipulant les exemples vous lancez par erreur un calcul trop long, l'interpréteur reste occupé jusqu'à en avoir fini avec ce calcul, et ne pourra pas évaluer d'autres cellules tant qu'il n'aura pas fini. Simulons ça avec un petit bout de programme qui attend bêtement pendant 5 secondes.

In [7]:
from time import sleep
sleep(5)
"c'est fini"

'fini'

Vous remarquez que pendant le temps du `sleep`, le nombre en face du label `In[]` est remplacé par une étoile, qui indique que votre interpréteur python est occupé.

Si cela vous arrive suite à une fausse manœuvre (vous lancez une boucle qui ne termine pas), ou si vous n'êtes pas assez patient pour attendre, pensez à faire, via les menus du notebook:
* *Kernel* → *Interrupt* pour interrompre le traitement, ou encore
* *Kernel* → *Restart* pour redémarrer votre interpréteur.

### Utilitaire `%%timeit`

À l'intérieur d'un notebook, lorsqu'on a besoin de mesurer le temps d'exécution d'un fragment de code, il est pratique d'utiliser l'astuce suivante:

In [ ]:
%%timeit
for i in range(10**3):
    j = i**2

La première ligne de cette cellule utilise ce qu'on appelle une *magic* ipython, et elle permet de mesurer avec le module `timeit` le temps d'exécution du code dans le reste de la cellule.

Dans mon environnement, après avoir évalué cette cellule le système m'affiche ceci:

```
310 µs ± 6.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
```

qui prend quelques secondes, car la mesure du temps est faite en exécutant le code plusieurs fois. Ici en l'occurrence le chiffre qui nous intéresse est le tout premier, qui nous dit que pour exécuter toute la boucle il nous a fallu en moyenne 310 µs.

##### `%%timeit -n`

Nous utiliserons parfois l'option `-n` qui nous permet de fixer le nombre de fois où le code est exécuté, pour fluidifier le notebook, cela se présente comme ceci:

In [3]:
%%timeit -n 10
# sans rien préciser ça prend vraiment longtemps
# ça vous donne l'occasion de faire un Kernel .. Interrupt :)
for i in range(10**5):
    j = i**2

33.1 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Le module `time`

Une autre façon de mesurer les temps d'exécution, plus basique mais moins fiable que `timeit`, consiste à utiliser la fonction `time.time()`, qui retourne l'heure de l'horloge interne, en secondes. On peut donc faire quelque chose comme:

In [8]:
from time import time

# on enregistre l'heure au début
debut = time()

# on fait un traitement
for i in range(10**5):
    j = i**2

# on enregistre l'heure à la fin
fin = time()

# on peut calculer la durée observée
print(f"Durée observée {fin-debut}")

Durée observée 0.04228615760803223


### La fonction `range()`

La fonction *builtin* `range`, qu'on a déjà rencontré plusieurs fois, permet d'itérer sur un intervalle d'entiers:

In [22]:
for i in range(3):
    print(i)

0
1
2


En fait elle prend jusqu'à trois arguments, et comme vous l'avez peut-être remarqué ou deviné, elle se comporte presque exactement comme les indices dans une slice, c'est-à dire:

In [16]:
s = "0123456789"

In [23]:
list(range(3, 7))

[3, 4, 5, 6]

In [21]:
s[3:7]

'3456'

In [25]:
list(range(3, 9, 2))

[3, 5, 7]

In [24]:
s[3:9:2]

'357'

In [26]:
list(range(9, 1, -2))

[9, 7, 5, 3]

In [29]:
s[9:1:-2]

'9753'

### Pourquoi `list(range())` ?

Vous pouvez vous demander pourquoi dans les exemples ci-dessus on a toujours appelé `list()` sur le résultat de `range()`, et voici pourquoi:

In [36]:
r = range(3)
r

range(0, 3)

Comme vous le voyez, le retour de la fonction est un objet de type `range`, qui est un itérateur et non pas une liste (comme c'était d'ailleurs le cas en python2).

Dans nos exemples, nous voulions voir le contenu de ce range, et un moyen simple pour faire cela est de passer l'itérateur à la fonction `list`, qui construit effectivement une liste contenant les objet énumérés par l'itérateur, et ainsi on peut les voir facilement.

La différence entre les deux approches, c'est-à-dire le fait que `range` renvoye un itérateur plutôt qu'une liste, est représentative, car de nombreuses autres fonctions *builtin* sont dans le même cas, comme par exemple:

In [47]:
z = zip ( ('a', 'b'), ('c', 'd'))
z

In [48]:
# qui contient
list(z)

[('a', 'c'), ('b', 'd')]

In [49]:
e = enumerate( ('a', 'b', 'c'))
e

In [51]:
# qui contient
list(e)

[]

C'est vrai aussi de pas mal de méthodes sur types *builtin*:

In [52]:
d = { 'a' : 'un', 'b': 'deux' }

In [53]:
k = d.keys()
k

dict_keys(['a', 'b'])

In [54]:
# qui contient
list(k)

['a', 'b']

### Pourquoi des itérateurs partout ?

Nous allons mesurer le temps qu'il faut pour **simplement construire** un objet liste, lorsque les tailles commencent à devenir substantielles:

In [55]:
# de 100.000 à 50 millions
tailles = [10**5, 10**6, 10**7, 5*10**7]

Voyons le temps que prend uniquement la **construction** d'une grosse liste.

In [60]:
import time

for taille in tailles:
    beg = time.time()
    liste = list(range(taille))
    end = time.time()
    print(f"Création de la liste de taille {taille}: {end-beg}s")

Création de la liste de taille 100000: 0.0025811195373535156s
Création de la liste de taille 1000000: 0.02711319923400879s
Création de la liste de taille 10000000: 0.28140783309936523s
Création de la liste de taille 50000000: 1.8900179862976074s


Si maintenant on construit un itérateur équivalent on mesure un temps beaucoup plus court:

In [61]:
import time

for taille in tailles:
    beg = time.time()
    iterateur = range(taille)
    end = time.time()
    print(f"Création de l'itérateur de taille {taille}: {end-beg}s")

Création de l'itérateur de taille 100000: 2.86102294921875e-06s
Création de l'itérateur de taille 1000000: 3.0994415283203125e-06s
Création de l'itérateur de taille 10000000: 1.1920928955078125e-06s
Création de l'itérateur de taille 50000000: 2.86102294921875e-06s


Vous pouvez remarquer que:
* les ordres de grandeur sont complètement différents
* en tendance, la **création d'un itérateur** de type `range` est quasiment **instantanée** quelle que soit la taille,
* alors que la création d'une liste équivalente prend **un temps beaucoup plus important**
* et d'autant plus long que la liste est grande.

Si on regarde la mémoire occupée (avec [sys.getsizeof](https://docs.python.org/3/library/sys.html#sys.getsizeof)) par les deux sortes d'objet, c'est également très différent:

In [65]:
import sys

for taille in tailles:
    liste = list(range(taille))
    print(f"liste de taille {taille} = {sys.getsizeof(liste)} bytes")

liste de taille 100000 = 900112 bytes
liste de taille 1000000 = 9000112 bytes
liste de taille 10000000 = 90000112 bytes
liste de taille 50000000 = 450000112 bytes


450000112

In [68]:
import sys

for taille in tailles:
    iterateur = range(taille)
    print(f"iterateur de taille {taille} = {sys.getsizeof(iterateur)} bytes")

iterateur de taille 100000 = 48 bytes
iterateur de taille 1000000 = 48 bytes
iterateur de taille 10000000 = 48 bytes
iterateur de taille 50000000 = 48 bytes


Là encore, on a une totale disproportion des ressources utilisées.

Or, et c'est le point important, la plupart du temps on **n'a pas du tout besoin** de cette liste de valeurs sur lesquelles itérer, car on la jette quasiment immédiatement, comme vous pouvez le voir si vous exécutez les deux animations suivantes dans pythontutor:

In [ ]:
%load_ext ipythontutor

In [79]:
%%ipythontutor curInstr=3
taille = 10
iterateur = range(taille)
counter = sum(iterateur)    

In [81]:
%%ipythontutor curInstr=3
taille = 10
liste = list(range(taille))
counter = sum(liste)    

### Ce qu'il faut retenir

Pour résumer ce complément, retenez que: 

 * la **construction d'une liste**, surtout si elle est très longue, peut avoir un **coût non négligeable** en temps, et aussi en mémoire;
 * c'est pourquoi il convient de s'efforcer de **ne créer une liste** que lorsque c'est **réellement nécessaire**;
 * et dans tous les autres cas - c'est à dire à chaque fois que la liste n'est qu'un **accessoire de calcul**, et ne représente pas une fin en soi - il faut **préférer** l'utilisation d'**itérateurs**.

## Complément - niveau intermédiaire

### Allouer et initialiser de la mémoire prend du temps

Ce phénomène peut vous paraître surprenant si vous n'êtes pas familier avec l'informatique. À première vue, si on juge superficiellement, on peut se demander ce qui se passe. 

En fait, pour créer la liste des `taille` premiers entiers, il faut
 * d'abord allouer suffisamment de mémoire pour tous les ranger
 * et ensuite remplir les `taille` cases de la liste avec les valeurs

Ces deux opérations semblent banales, mais elles prennent néanmoins un peu de temps, qui à grande échelle devient sensible, comme nous venons de l'expérimenter.

### Un itérateur est un objet minuscule

A contrario, un itérateur du type `range` ne **contient presque rien**. Cela sera approfondi en semaine 6, mais pour anticiper un peu la fonction d'un iterateur `range` consiste uniquement à mémoriser les paramètres de la boucle, et à quelle étape on en est rendu à un moment donné. 

Ce qui explique le temps très faible, et constant en fonction de `taille`, que l'on a observé pour la création de nos itérateurs.

## Complément - niveau avancé

Pour finir, et pour revenir sur les mesures de performances, voici une astuce qui permet de lancer  de petits benchmarkes dans un terminal&nbsp;:

    $ python -m timeit 'liste=range(10**6)' 'for x in liste: x+1'
    10 loops, best of 3: 50.5 msec per loop

Ceci met en jeu un certain nombre de choses nouvelles:
 * python avec l'option -m permet d'importer un module, en l'occurrence ici [le module `timeit`](https://docs.python.org/3/library/timeit.html);
 * avec cette forme on peut passer à `timeit` plusieurs instructions; ici nous avons deux instructions, une pour initialiser `liste`, la seconde pour lancer la boucle `for`;
 * il est possible d'écrire des instructions sur une seule ligne. Ici le dernier argument passé à python est 
 
    for x in liste: x+1
    
   qui est interprété comme une seule ligne. Cette pratique doit absolument rester limitée à de tels usages spécifiques.


Cette forme est pratique notamment parce que `timeit` fait, comme on le voit, plusieurs essais successifs qui donnent un résultat plus représentatif. C'est pourquoi vous la trouverez fréquemment utilisée dans les forums de discussion autour de python.